In [2]:
from typing import Optional

import peft
import torch
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, HfArgumentParser


In [4]:
peft_model_id = "tloen/alpaca-lora-7b"
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    return_dict=True,
    torch_dtype=torch.float16,
)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)
model.eval()


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:23<00:00,  1.43it/s]


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): Dropout(p=0.05, inplace=False)
                (lora_A): Linear(in_features=4096, out_features=16, bias=False)
                (lora_B): Linear(in_features=16, out_features=4096, bias=False)
              )
              (k_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): Dropout(p=0.05, inplace=False)
                (lora_A): Linear(in_features=4096, out_features=16, bias=False)
                (lora_B): Linear(in_features=16, out_features=4096, bias=False)
              )
              (v_proj): Linear(
                in_f

In [5]:
key_list = [key for key, _ in model.base_model.model.named_modules() if "lora" not in key]

In [6]:
key_list

['',
 'model',
 'model.embed_tokens',
 'model.layers',
 'model.layers.0',
 'model.layers.0.self_attn',
 'model.layers.0.self_attn.q_proj',
 'model.layers.0.self_attn.k_proj',
 'model.layers.0.self_attn.v_proj',
 'model.layers.0.self_attn.o_proj',
 'model.layers.0.self_attn.rotary_emb',
 'model.layers.0.mlp',
 'model.layers.0.mlp.gate_proj',
 'model.layers.0.mlp.down_proj',
 'model.layers.0.mlp.up_proj',
 'model.layers.0.mlp.act_fn',
 'model.layers.0.input_layernorm',
 'model.layers.0.post_attention_layernorm',
 'model.layers.1',
 'model.layers.1.self_attn',
 'model.layers.1.self_attn.q_proj',
 'model.layers.1.self_attn.k_proj',
 'model.layers.1.self_attn.v_proj',
 'model.layers.1.self_attn.o_proj',
 'model.layers.1.self_attn.rotary_emb',
 'model.layers.1.mlp',
 'model.layers.1.mlp.gate_proj',
 'model.layers.1.mlp.down_proj',
 'model.layers.1.mlp.up_proj',
 'model.layers.1.mlp.act_fn',
 'model.layers.1.input_layernorm',
 'model.layers.1.post_attention_layernorm',
 'model.layers.2',
 'mo

In [8]:
for key in key_list:
    print(key)
    parent, target, target_name = model.base_model._get_submodules(key)
    assert not isinstance(target, peft.tuners.lora.MergedLinear)
    if isinstance(target, peft.tuners.lora.Linear):
        break
        bias = target.bias is not None
        new_module = torch.nn.Linear(target.in_features, target.out_features, bias=bias)
        model.base_model._replace_module(parent, target_name, new_module, target)
        




model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj


In [29]:
target??

In [28]:
target.weight??

In [16]:
a = list(target.modules())

In [26]:
model.base_model._replace_module??

In [22]:
list(a[-1].parameters())

[Parameter containing:
 tensor([[-4.4180e-03,  1.1155e-02, -3.3248e-02,  ..., -1.3536e-02,
          -2.2070e-02,  3.1977e-02],
         [ 4.4443e-04, -4.3023e-03,  1.2348e-02,  ...,  9.3919e-03,
          -2.9612e-03, -1.7978e-02],
         [ 1.5016e-02, -1.0953e-02,  9.2405e-03,  ...,  9.6885e-03,
           1.4855e-02, -9.0018e-03],
         ...,
         [ 9.9188e-03, -2.0773e-03, -1.8999e-03,  ...,  8.0707e-03,
           7.2097e-05, -3.8060e-03],
         [ 1.1379e-02, -6.9277e-03,  2.5411e-03,  ...,  7.7655e-03,
           2.8084e-03, -4.0624e-03],
         [ 2.3009e-02, -1.4978e-02,  3.6532e-02,  ...,  2.0317e-02,
           1.9169e-02, -3.7267e-02]], requires_grad=True)]

In [ ]:
model = model.base_model.model

In [8]:
model.push_to_hub(
    f"alexrame/alpaca-lora-7b-merged",
    use_auth_token=True)



pytorch_model-00002-of-00002.bin:   0%|                                                                                                                                                                                         | 0.00/3.50G [00:00<?, ?B/s]
pytorch_model-00002-of-00002.bin:   0%|                                                                                                                                                                             | 8.19k/3.50G [00:00<31:34:20, 30.8kB/s]

pytorch_model-00002-of-00002.bin:   0%|                                                                                                                                                                                | 1.56M/3.50G [00:00<11:30, 5.07MB/s]

pytorch_model-00001-of-00002.bin:   0%|                                                                                                                                                                                | 2.48M/9.98G [00:00<2

pytorch_model-00002-of-00002.bin:   2%|████▎                                                                                                                                                                           | 87.0M/3.50G [00:07<05:29, 10.3MB/s]

pytorch_model-00001-of-00002.bin:   1%|█▌                                                                                                                                                                              | 89.4M/9.98G [00:07<12:11, 13.5MB/s]

pytorch_model-00002-of-00002.bin:   3%|████▌                                                                                                                                                                           | 90.4M/3.50G [00:07<04:26, 12.8MB/s]

pytorch_model-00002-of-00002.bin:   3%|████▉                                                                                                                                                                           | 97.1M/3.50G [00:08<06

pytorch_model-00002-of-00002.bin:   5%|█████████▎                                                                                                                                                                       | 184M/3.50G [00:15<04:05, 13.5MB/s]

pytorch_model-00002-of-00002.bin:   5%|█████████▍                                                                                                                                                                       | 187M/3.50G [00:15<04:04, 13.5MB/s]

pytorch_model-00001-of-00002.bin:   2%|███▎                                                                                                                                                                             | 184M/9.98G [00:15<13:30, 12.1MB/s]

pytorch_model-00001-of-00002.bin:   2%|███▎                                                                                                                                                                             | 186M/9.98G [00:15<13

pytorch_model-00002-of-00002.bin:   8%|██████████████▌                                                                                                                                                                  | 288M/3.50G [00:23<02:57, 18.0MB/s]

pytorch_model-00001-of-00002.bin:   3%|████▉                                                                                                                                                                            | 281M/9.98G [00:23<12:12, 13.2MB/s]

pytorch_model-00002-of-00002.bin:   8%|██████████████▊                                                                                                                                                                  | 293M/3.50G [00:24<04:10, 12.8MB/s]

pytorch_model-00002-of-00002.bin:   9%|███████████████                                                                                                                                                                  | 299M/3.50G [00:24<03

pytorch_model-00001-of-00002.bin:   4%|██████▍                                                                                                                                                                         | 368M/9.98G [00:46<4:38:00, 576kB/s]

pytorch_model-00001-of-00002.bin:   4%|██████▌                                                                                                                                                                         | 369M/9.98G [00:46<3:53:34, 685kB/s]

pytorch_model-00001-of-00002.bin:   4%|██████▌                                                                                                                                                                        | 374M/9.98G [00:46<2:08:58, 1.24MB/s]

pytorch_model-00001-of-00002.bin:   4%|██████▌                                                                                                                                                                        | 376M/9.98G [00:46<1:39

...

pytorch_model-00001-of-00002.bin:  96%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋      | 9.62G/9.98G [14:10<00:39, 9.19MB/s]

pytorch_model-00001-of-00002.bin:  96%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋      | 9.62G/9.98G [14:10<00:37, 9.62MB/s]

pytorch_model-00001-of-00002.bin:  96%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋      | 9.62G/9.98G [14:10<00:26, 13.4MB/s]

pytorch_model-00001-of-00002.bin:  96%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊      | 9.62G/9.98G [14:10<00

pytorch_model-00001-of-00002.bin:  97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎    | 9.71G/9.98G [14:17<00:16, 16.6MB/s]

pytorch_model-00001-of-00002.bin:  97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎    | 9.71G/9.98G [14:18<00:31, 8.50MB/s]

pytorch_model-00001-of-00002.bin:  97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎    | 9.71G/9.98G [14:18<00:29, 8.99MB/s]

pytorch_model-00001-of-00002.bin:  97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍    | 9.72G/9.98G [14:18<00

pytorch_model-00001-of-00002.bin:  98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉   | 9.80G/9.98G [14:25<00:12, 14.3MB/s]

pytorch_model-00001-of-00002.bin:  98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉   | 9.81G/9.98G [14:25<00:10, 15.6MB/s]

pytorch_model-00001-of-00002.bin:  98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████   | 9.81G/9.98G [14:26<00:18, 9.11MB/s]

pytorch_model-00001-of-00002.bin:  98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████   | 9.81G/9.98G [14:26<00

pytorch_model-00001-of-00002.bin:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 9.90G/9.98G [14:33<00:04, 15.7MB/s]

pytorch_model-00001-of-00002.bin:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 9.90G/9.98G [14:34<00:08, 8.46MB/s]

pytorch_model-00001-of-00002.bin:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 9.91G/9.98G [14:34<00:06, 11.3MB/s]

pytorch_model-00001-of-00002.bin:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 9.91G/9.98G [14:34<00

CommitInfo(commit_url='https://huggingface.co/alexrame/alpaca-lora-7b-merged/commit/d46ed85e677a450bee66260bfe98b3161dc8237e', commit_message='Upload LlamaForCausalLM', commit_description='', oid='d46ed85e677a450bee66260bfe98b3161dc8237e', pr_url=None, pr_revision=None, pr_num=None)

In [11]:
model.base_model

LlamaModel(
  (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
  (layers): ModuleList(
    (0): LlamaDecoderLayer(
      (self_attn): LlamaAttention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (rotary_emb): LlamaRotaryEmbedding()
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
        (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
        (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
        (act_fn): SiLUActivation()
      )
      (input_layernorm): LlamaRMSNorm()
      (post_attention_layernorm): LlamaRMSNorm()
    )
    (1): LlamaDecoderLayer(
      (self_attn): LlamaAttention(
        (q_proj): Linear(in_features